In [9]:
# Import libraries
import betfairlightweight
from betfairlightweight import filters
import pandas as pd
import numpy as np
import os
import datetime
import json
from utils import process_runner_books
import pytz


# Change this certs path to wherever you're storing your certificates
certs_path = "/home/naing/certs/"

# Change these login details to your own
my_username = "mightytranks@gmail.com"
my_password = "OsGiwuevzRf4925Lfty!"
# my_app_key = "X9EGwt5zbpg8o5xk" #demo
my_app_key = "XzTt9hf8DAsmOuUa" #live

trading = betfairlightweight.APIClient(username=my_username,
                                       password=my_password,
                                       app_key=my_app_key,
                                       certs=certs_path)

trading.login()


<LoginResource>

In [10]:
gh_racing_id = 4339 #Greyhound racing ID
# Filter for just greyhoud racing
greyhound_racing_filter = filters.market_filter(
    event_type_ids=[gh_racing_id],
    market_countries=['GB'],
    market_type_codes=['FORECAST'],
    market_start_time={
        'to': (datetime.datetime.utcnow() + datetime.timedelta(hours=6)).strftime("%Y-%m-%dT%TZ") #sydtime 5pm to 7am next day for GB gh races
    }
)
greyhound_racing_filter


{'eventTypeIds': [4339],
 'marketCountries': ['GB'],
 'marketTypeCodes': ['FORECAST'],
 'marketStartTime': {'to': '2022-06-19T19:47:22Z'}}

In [11]:
#  This returns a list
gb_gh_events = trading.betting.list_events(
    filter=greyhound_racing_filter)

# Create a DataFrame with all the events by iterating over each event object
gb_gh_events_df = pd.DataFrame({
    'Event Name': [event_object.event.name for event_object in gb_gh_events],
    'Event ID': [event_object.event.id for event_object in gb_gh_events],
    'Event Venue': [event_object.event.venue for event_object in gb_gh_events],
    'Country Code': [event_object.event.country_code for event_object in gb_gh_events],
    'Time Zone': [event_object.event.time_zone for event_object in gb_gh_events],
    'Open Date': [event_object.event.open_date for event_object in gb_gh_events],
    'Market Count': [event_object.market_count for event_object in gb_gh_events]
})

gb_gh_events_df

,Event Name,Event ID,Event Venue,Country Code,Time Zone,Open Date,Market Count
0,Swindon (F/C) 19th Jun,31536112,Swindon,GB,Europe/London,2022-06-19 17:19:00,10
1,Kinsley (F/C) 19th Jun,31536083,Kinsley,GB,Europe/London,2022-06-19 13:04:00,9
2,Henlow (F/C) 19th Jun,31536131,Henlow,GB,Europe/London,2022-06-19 17:17:00,9
3,Central Park (F/C) 19th Jun,31536125,Central Park,GB,Europe/London,2022-06-19 17:09:00,9
4,Towcester (F/C) 19th Jun,31536095,Towcester,GB,Europe/London,2022-06-19 12:57:00,11


In [12]:
#TODO: extract a list of IDs for the forecast markets
fc_venue_ids = []
fc_venue_names = []
for eventObj in gb_gh_events:
    fc_venue_ids.append(eventObj.event.id)
    fc_venue_names.append(eventObj.event.name)

fc_venue_ids, fc_venue_names


(['31536112', '31536083', '31536131', '31536125', '31536095'],
 ['Swindon (F/C) 19th Jun',
  'Kinsley (F/C) 19th Jun',
  'Henlow (F/C) 19th Jun',
  'Central Park (F/C) 19th Jun',
  'Towcester (F/C) 19th Jun'])

In [13]:
# market_catalogue_filter = filters.market_filter(event_ids=[venueOfTheDay_id])
market_catalogue_filter = filters.market_filter(
    event_ids=fc_venue_ids)

market_catalogues = trading.betting.list_market_catalogue(
    filter=market_catalogue_filter,
    market_projection= ['MARKET_START_TIME', 'EVENT'],
    max_results='5',
    sort='FIRST_TO_START'
)

market_types_venueOfTheDay_df = pd.DataFrame({
    'Market Name': [market_cat_object.market_name for market_cat_object in market_catalogues],
    'Market ID': [market_cat_object.market_id for market_cat_object in market_catalogues],
    'Market Start Time': [market_cat_object.market_start_time for market_cat_object in market_catalogues],
    'Total Matched': [market_cat_object.total_matched for market_cat_object in market_catalogues],
    'Venue': [market_cat_object.event.venue for market_cat_object in market_catalogues]
    
})

market_types_venueOfTheDay_df

,Market Name,Market ID,Market Start Time,Total Matched,Venue
0,Forecast,1.200287435,2022-06-19 13:52:00,17.69000,Towcester
1,Forecast,1.200287392,2022-06-19 13:54:00,0.00000,Kinsley
2,Forecast,1.200287393,2022-06-19 14:09:00,86.46872,Kinsley
3,Forecast,1.200287436,2022-06-19 14:11:00,90.89122,Towcester
4,Forecast,1.200287394,2022-06-19 14:26:00,0.00000,Kinsley


In [15]:
timeNow = (datetime.datetime.now(pytz.timezone("Europe/London"))-datetime.timedelta(hours=1)) ## minus 1 hr due to daylight savings maybe?
print(timeNow.strftime("%Y-%m-%d %T"))
## TODO Compare the list of markets - if timeNow is less than market start time (ie. market hasn't started yet), pick it. Otherwise, skip
time1 = timeNow.replace(tzinfo=pytz.UTC)

for marketObj in market_catalogues:
    time2 = marketObj.market_start_time.replace(tzinfo=pytz.UTC)
    if time1 < time2:
        print("Found the market to lay: Name = " + marketObj.event.venue + " id = " + str(marketObj.market_id))
        break



2022-06-19 13:48:22
Found the market to lay: Name = Towcester id = 1.200287435


In [ ]:
# ###
# # Get odds from NEDS
# ###
# from utils import choose_lay_option

# venueName = 'doncaster'
# lay_option = choose_lay_option(venueName)
# if(lay_option == -1):
#     print("ERROR!!! 2 same odds found")
    
# elif(lay_option == -2):
#     print("ERROR!!! less than 6 runners")

# else:
#     print("Lay option found successfully")
#     print(lay_option)

In [21]:
#TODO: extract a list of IDs for the forecast markets
fc_race_ids = []
fc_race_names = []
for raceObj in market_catalogues:
    fc_race_ids.append(raceObj.market_id)

fc_race_ids

['1.200268737',
 '1.200268738',
 '1.200268739',
 '1.200270275',
 '1.200270276',
 '1.200270277',
 '1.200270278',
 '1.200270279',
 '1.200268788',
 '1.200268964']

In [22]:
# pick the random race id
# random_pick = random.randint(1,len(fc_race_ids))
# myRaceID = fc_race_ids[0]
myRaceID = '1.200268737'

In [23]:
# Create a price filter. Get all traded and offer data
price_filter = filters.price_projection(
    price_data=['EX_BEST_OFFERS']
)

# Request market books
market_books = trading.betting.list_market_book(
    market_ids=[myRaceID],
    price_projection=price_filter
)

# Grab the first market book from the returned list as we only requested one market 
market_book = market_books[0]

availableLays_df = pd.DataFrame({
    'keyLineDescription': [market_obj.key_line_description for market_obj in market_books],
    'Num_active_runners': [market_obj.number_of_runners for market_obj in market_books],
})
availableLays_df

# runners_df = process_runner_books(market_book.runners)
# runners_df

,keyLineDescription,Num_active_runners
0,None,30


In [ ]:
fav_selection_id = runners_df.loc[runners_df['Best Lay Price'].idxmin(), 'Selection ID']
fav_price = runners_df.loc[runners_df['Best Lay Price'].idxmin(), 'Best Lay Price']

limit_order_filter = filters.limit_order(
    size=5,
    price=3,
    persistence_type='LAPSE'
)

instructions_filter = filters.place_instruction(
    selection_id = str(fav_selection_id),
    order_type = "LIMIT",
    side="LAY",
    limit_order=limit_order_filter
)

instructions_filter

{'orderType': 'LIMIT',
 'selectionId': '39483384',
 'side': 'LAY',
 'limitOrder': {'price': 3, 'persistenceType': 'LAPSE', 'size': 5}}

In [ ]:
# order = trading.betting.place_orders(
#     market_id = myRaceID,
#     customer_strategy_ref='Naing_maker',
#     instructions=[instructions_filter]
# )

In [ ]:
# order.__dict__

In [ ]:
# trading.betting.list_current_orders(customer_strategy_refs=['Naing_maker']).__dict__

In [ ]:
# cancelled_order = trading.betting.cancel_orders(market_id=myRaceID)

In [ ]:
# # listClearedOrders
# cleared_orders = trading.betting.list_cleared_orders(bet_status="CANCELLED",
#                                                     market_ids=[myRaceID])
# # Create a DataFrame from the orders
# pd.DataFrame(cleared_orders._data['cancelledOrders'])
# cleared_orders